In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Environment Setup for Exploring ZeitGhost Backend

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/referencearchitectures/setup.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/referencearchitectures/setup.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/language/referencearchitectures/setup.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>

## Overview

<center>
<img src="imgs/zghost_overview.png" width="1200"/>
</center>
In order to build out the conversational agent which can use GDELT data and BigQuery public trends data to answer natural language questions, we need to first investigate what a suitable ACTOR_NAME for extracting the GDELT data could be, along with creating necessary cloud resources needed to build the architecture. 

Additionally, this notebook will save a persistent configuration file that you can reuse throughout the rest of the notebooks so the variables will be parameterized as you continue to work through the notebooks. 

### Get started with language models
To view more information about getting started with language models on Vertex AI, see [Getting Started with the Vertex AI PaLM API & Python SDK](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb)

### Environment Setup
This notebook can be used to create configurations once that can be used for the rest of the notebooks to create the ZeitGhost Backend:
1. [Setup Vertex Vector Store](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb)
2. [GDELT DataOps](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb)
3. [Vector Store Index Loader](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb) and 3a [Optional - Chunk up the Docs](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb)
4. [Build Zeitghost Image](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb)
5. [GDELT Pipelines](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb)
6. [Plan and Execute Agents](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb)

For more information, check out the [documentation on generative AI support for Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview).

### Costs
This tutorial uses billable components of Google Cloud:

* Vertex AI Generative AI Studio

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing),
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Data security
**Q: Does Google use customer data to improve its foundation models?**  
A: No, Google does not use customer data to improve foundation models. Customer data is only used to generate a response from the model.

**Q: Do Google employees see data that I submit to the model?**  
A: No, Google employees have no access to customer data and all data is encrypted in-transit, in-use, and at-rest. 

**Q: Does Google store any of the customer data that is sent to the model?**  
A: No, Google does not store customer data. However, Google may temporarily cache customer data for the duration of the request, such as prompt tuning pipeline and batch prediction. 

**Q: Does Google log data?**  
A: No, Google does not log customer data. System-level logs help Google ensure system health and availability.

### Responsible AI
Large language models (LLMs) can translate language, summarize text, generate creative writing, generate code, power chatbots and virtual assistants, and complement search engines and recommendation systems. At the same time, as an early-stage technology, its evolving capabilities and uses create potential for misapplication, misuse, and unintended or unforeseen consequences. Large language models can generate output that you don't expect, including text that's offensive, insensitive, or factually incorrect.

What's more, the incredible versatility of LLMs is also what makes it difficult to predict exactly what kinds of unintended or unforeseen outputs they might produce. Given these risks and complexities, the PaLM API is designed with [Google's AI Principles](https://ai.google/principles/) in mind. However, it is important for developers to understand and test their models to deploy safely and responsibly. To aid developers, the Generative AI Studio has built-in content filtering, and the PaLM API has safety attribute scoring to help customers test Google's safety filters and define confidence thresholds that are right for their use case and business. Please refer to the [Safety filters and attributes](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/responsible-ai#safety_filters_and_attributes) section to learn more.

When the PaLM API is integrated into a customer's unique use case and context, additional responsible AI considerations and [PaLM limitations](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/responsible-ai#palm_limitations) may need to be considered. We encourage customers to leverage fairness, interpretability, privacy and security [recommended practices](https://ai.google/responsibilities/responsible-ai-practices/).

## Getting started

### Install Packages

Run `pip` requirements.txt in either (1) the notebook cell below or (2) in a notebook terminal window

In [5]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

!pip install --no-cache-dir -r ./requirements.txt --user -q

### **IMPORTANT** Restart the Kernel

**Colab only:** Uncomment the following cell to restart the kernel. For Vertex AI Workbench you can restart the terminal using the button on top. 

In [2]:
# # Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### Authenticating your notebook environment
* If you are using **Colab** to run this notebook, uncomment the cell below and continue.
* If you are using **Vertex AI Workbench**, check out the setup instructions [here](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env).

In [3]:
# from google.colab import auth
# auth.authenticate_user()

### Import libraries

In [1]:
import pandas as pd
import numpy as np
import json
import uuid

from google.cloud import aiplatform as vertex_ai
from google.cloud import storage
from google.cloud import bigquery

## Set Environment variables

* `PROJECT_NUM`         - manually enter (TODO: investigating single GCP `PROJECT_ID` to have many `PROJECT_NUM`s?)
* `LOCATION` & `REGION` - location for GCS and Vertex AI assets
* `BQ_LOCATION`         - location for BigQuery tables

In [2]:
GCP_PROJECTS             = !gcloud config get-value project
PROJECT_ID               = GCP_PROJECTS[0]

PROJECT_NUM              = !gcloud projects describe $PROJECT_ID --format="value(projectNumber)"
PROJECT_NUM              = PROJECT_NUM[0]

# locations / regions for cloud resources
LOCATION                 = 'us-central1'        # TODO
REGION                   = LOCATION             # TODO
BQ_LOCATION              = 'US'                 # TODO

# VPC network (TODO: public endpoints)
VPC_NETWORK_NAME         = "me-network" # TODO

print(f"PROJECT_ID       : {PROJECT_ID}")
print(f"PROJECT_NUM      : {PROJECT_NUM}")
print(f"LOCATION         : {LOCATION}")
print(f"REGION           : {REGION}")
print(f"BQ_LOCATION      : {BQ_LOCATION}")
print(f"VPC_NETWORK_NAME : {VPC_NETWORK_NAME}")

PROJECT_ID       : wortz-project-352116
PROJECT_NUM      : 679926387543
LOCATION         : us-central1
REGION           : us-central1
BQ_LOCATION      : US
VPC_NETWORK_NAME : me-network


### Initialize Google Cloud SDK Clients

In [3]:
# cloud storage client
storage_client = storage.Client(project=PROJECT_ID)

# Vertex client
vertex_ai.init(project=PROJECT_ID, location=LOCATION)

# bigquery client
bqclient = bigquery.Client(
    project=PROJECT_ID,
    # location=LOCATION
)

### Create new assets or use existing

**`CREATE_NEW_ASSETS`**
* `True` creates new GCS buckets, BQ tables, Matching Engine `Indexes` and `IndexEndpoints`
* `False` uses existing 

**asset display names** use combination of the following variables:
* `ACTOR_NAME`     - string, all lower case (e.g., "google cloud") representing the organization or company of interest. Used in BigQuery Public Dataset queries. See the [GDELT Event Cookbook](http://data.gdeltproject.org/documentation/GDELT-Event_Codebook-V2.0.pdf) for detailed documentation.
* `ACTOR_PREFIX`   - short customer prefix (e.g., `gcp`), for naming artifacts and GCP resources
* `ACTOR_CATEGORY` - industry or category for the actor. used in LLM agent Q/A (e.g.,"What will the impact of inflation have on the `{ACTOR_CATEGORY}` category over the next 6 months?")
* `VERSION`        - for development purposes and standardizing naming convention; use `VERSION` throughout notebooks to create multiple versions of assets, without having to create all new assets each time you want to run the code.

In [4]:
# create new BQ datasets, tables, etc.?
CREATE_NEW_ASSETS         = True            # TODO: True | False

# index IDs
ACTOR_NAME                = 'google'         # TODO
ACTOR_PREFIX              = 'ggl'            # TODO
ACTOR_CATEGORY            = 'technology' # TODO
VERSION                   = 'v1'             # TODO

print(f"CREATE_NEW_ASSETS : {CREATE_NEW_ASSETS}")
print(f"ACTOR_PREFIX      : {ACTOR_PREFIX}")
print(f"VERSION           : {VERSION}")
print(f"ACTOR_NAME        : {ACTOR_NAME}")
print(f"ACTOR_CATEGORY    : {ACTOR_CATEGORY}")

CREATE_NEW_ASSETS : True
ACTOR_PREFIX      : ggl
VERSION           : v1
ACTOR_NAME        : google
ACTOR_CATEGORY    : technology


## Test `ACTOR_NAME` in GDELT

Before continuing to create the naming convention we will use throughout these notebooks, confirm your actor (`ACTOR_NAME`) can be found in both GDELT tables below.

The GDELT queries look for articles between two dates specified by the user: (`min_date`, `max_date`)
* Start small to avoid creating a massive dataframe in-notebook. To scale to larger time periods, you can use the pipeline functionality to orchestrate larger data extractions.
* Increase the time window as needed
* Ideally you can find 100s - 1000s of articles within the last few years. Some actors will meet this goal within a time window of a few days - great!

In [5]:
import sys
sys.path.append("..")
from zeitghost.gdelt.GdeltData import GdeltData
from zeitghost.bigquery.BigQueryAccessor import BigQueryAccessor

### Test GDELT events table
Set the date range you want to explore

In [6]:
# set dates - '%Y%m%d'
MIN_DATE = "2023-06-14" # TODO
MAX_DATE = "2023-06-15" # TODO

print(f"MIN_DATE     : {MIN_DATE}")
print(f"MAX_DATE     : {MAX_DATE}")

MIN_DATE     : 2023-06-14
MAX_DATE     : 2023-06-15


Pass the actor name and date range to see which kinds of data for your selection is available from the GDELT events table, and preview it in a pandas dataframe

In [7]:
events_data_accessor = BigQueryAccessor(
    PROJECT_ID
    , gdelt_project_id = 'gdelt-bq'
    , gdelt_dataset_id = 'gdeltv2'
    , gdelt_table_name = 'events'
)

# extract dataframe
gdelt_events_accessor = events_data_accessor.get_records_from_actor_keyword_df(
    keyword = ACTOR_NAME
    , min_date = MIN_DATE
    , max_date = MAX_DATE
)

print(gdelt_events_accessor.shape)
gdelt_events_accessor.head(1)

(510, 11)


,SQLDATE,new_date,Actor1Name,Actor2Name,GoldsteinScale,NumMentions,NumSources,NumArticles,AvgTone,SOURCEURL,url
0,20230615,2023-06-15,None,GOOGLE,-5.0,10,1,10,-2.29249,https://news.yahoo.com/eu-files-antitrust-char...,https://news.yahoo.com/eu-files-antitrust-char...


### Test GDELT entity table
Set the date range you want to explore

In [8]:
# set dates - '%Y%m%d'
MIN_DATE = "2023-01-14" # TODO
MAX_DATE = "2023-06-15" # TODO

print(f"MIN_DATE     : {MIN_DATE}")
print(f"MAX_DATE     : {MAX_DATE}")

MIN_DATE     : 2023-01-14
MAX_DATE     : 2023-06-15


Pass the actor name and date range to see which kinds of data for your selection is available from the GDELT global entity table, and preview it in a pandas dataframe

In [9]:
geg_data_accessor = BigQueryAccessor(
    PROJECT_ID
    , gdelt_project_id='gdelt-bq'
    , gdelt_dataset_id='gdeltv2'
    , gdelt_table_name='geg_gcnlapi'
)

geg_articles_accessor = geg_data_accessor.get_geg_article_data_v2_full_df(
    entity = ACTOR_NAME
    , min_date = MIN_DATE
    , max_date = MAX_DATE
)

print(geg_articles_accessor.shape)
geg_articles_accessor.head(1)

(29699, 3)


,url,date,avgSalience
0,https://gaceta.es/espana/sanchez-cambia-el-tab...,2023-05-30 05:46:42+00:00,0.167318


## Define Cloud Resource Names and Args

After confirming your `ACTOR_NAME` can be found in both GDELT tables, proceed to set the resource names and args below.

> Note: No need to edit the cell below. These values will be saved to a config file in GCS and loaded at the beginning of each subsequent notebook. This is intended to help keep track of the many assets used throughout these notebooks

In [10]:
# staging google cloud storage bucket
BUCKET_NAME              = f'zghost-{ACTOR_PREFIX}-{VERSION}-{PROJECT_ID}'
BUCKET_URI               = f'gs://{BUCKET_NAME}'

# bucket to stash embedding files
EMBEDDING_DIR_BUCKET     = f'{BUCKET_NAME}-emd-dir'
EMBEDDING_DIR_BUCKET_URI = f'gs://{EMBEDDING_DIR_BUCKET}'

# vpc network
VPC_NETWORK_FULL         = f"projects/{PROJECT_NUM}/global/networks/{VPC_NETWORK_NAME}"

# matching engine vector store
ME_INDEX_NAME            = f"vectorstore_{ACTOR_PREFIX}_{VERSION}"
ME_DIMENSIONS            = 768 # when using Vertex PaLM Embedding

# bigquery dataset
MY_BQ_DATASET            = BUCKET_NAME.lower().replace(PROJECT_ID,"").replace("-","_").rstrip("_")
MY_BQ_TRENDS_DATASET     = f"{MY_BQ_DATASET}_trends"

print(f"BUCKET_NAME               : {BUCKET_NAME}")
print(f"BUCKET_URI                : {BUCKET_URI}")
print(f"EMBEDDING_DIR_BUCKET_URI  : {EMBEDDING_DIR_BUCKET_URI}\n")
print(f"VPC_NETWORK_FULL          : {VPC_NETWORK_FULL}")
print(f"ME_INDEX_NAME             : {ME_INDEX_NAME}")
print(f"ME_DIMENSIONS             : {ME_DIMENSIONS}")
print(f"MY_BQ_DATASET             : {MY_BQ_DATASET}")
print(f"MY_BQ_TRENDS_DATASET      : {MY_BQ_TRENDS_DATASET}")

BUCKET_NAME               : zghost-ggl-v1-wortz-project-352116
BUCKET_URI                : gs://zghost-ggl-v1-wortz-project-352116
EMBEDDING_DIR_BUCKET_URI  : gs://zghost-ggl-v1-wortz-project-352116-emd-dir

VPC_NETWORK_FULL          : projects/679926387543/global/networks/me-network
ME_INDEX_NAME             : vectorstore_ggl_v1
ME_DIMENSIONS             : 768
MY_BQ_DATASET             : zghost_ggl_v1
MY_BQ_TRENDS_DATASET      : zghost_ggl_v1_trends


## Create Cloud Resources needed for these notebooks

> Note: you will only need to do this once per `ACTOR_NAME` + `VERSION`

### Create new GCS buckets

In [57]:
if CREATE_NEW_ASSETS:
    ! gsutil mb -l $LOCATION $BUCKET_URI
    ! gsutil mb -l $LOCATION $EMBEDDING_DIR_BUCKET_URI

Creating gs://zghost-way-v1-wortz-project-352116/...
ServiceException: 409 A Cloud Storage bucket named 'zghost-way-v1-wortz-project-352116' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.
Creating gs://zghost-way-v1-wortz-project-352116-emd-dir/...
ServiceException: 409 A Cloud Storage bucket named 'zghost-way-v1-wortz-project-352116-emd-dir' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [11]:
if CREATE_NEW_ASSETS:
    
    # dummy embedding
    init_embedding = {
        "id": str(uuid.uuid4()),
        "embedding": list(np.zeros(ME_DIMENSIONS))
    }

    # dump embedding to a local file
    with open("embeddings_0.json", "w") as f:
        json.dump(init_embedding, f)

    # write embedding to Cloud Storage
    ! gsutil cp embeddings_0.json {EMBEDDING_DIR_BUCKET_URI}/init_index/embeddings_0.json

Copying file://embeddings_0.json [Content-Type=application/json]...
/ [1 files][  3.8 KiB/  3.8 KiB]                                                
Operation completed over 1 objects/3.8 KiB.                                      


### Create new BigQuery datasets

For more information about working with the BigQuery Python SDK, see [SDK reference](https://cloud.google.com/python/docs/reference/bigquery/latest/index.html#google.cloud.bigquery.dataset.Dataset)

Create two BQ datasets:
* `MY_BQ_DATASET` will be used to store GDELT data of interest, including scraped article content
* `MY_BQ_TRENDS_DATASET` will be used to copy the Google Trends public dataset, which will later be used when we combine the decision making ability of LLMs with tools in order to create a system that can execute and implement solutions (i.e., [agents](https://python.langchain.com/en/latest/use_cases/personal_assistants.html))

In [ ]:
if CREATE_NEW_ASSETS:
    ds = bigquery.Dataset(f"{PROJECT_ID}.{MY_BQ_DATASET}")
    ds.location = 'us' #Multi-region is REGION[0:2]
    ds = bqclient.create_dataset(dataset = ds, exists_ok = False)

    print(ds.full_dataset_id)

In [ ]:
if CREATE_NEW_ASSETS:
    ds = bigquery.Dataset(f"{PROJECT_ID}.{MY_BQ_TRENDS_DATASET}")
    ds.location = 'us' #Multi-region is REGION[0:2]
    ds = bqclient.create_dataset(dataset = ds, exists_ok = False)

    print(ds.full_dataset_id)

### Save Notebook Configuration Data

If you want to avoid having to re-enter these across notebooks

In [14]:
config = f"""
PROJECT_ID               = \"{PROJECT_ID}\"
PROJECT_NUM              = \"{PROJECT_NUM}\"
LOCATION                 = \"{LOCATION}\"

REGION                   = \"{REGION}\"
BQ_LOCATION              = \"{BQ_LOCATION}\"
VPC_NETWORK_NAME         = \"{VPC_NETWORK_NAME}\"

CREATE_NEW_ASSETS        = \"{CREATE_NEW_ASSETS}\"
ACTOR_PREFIX             = \"{ACTOR_PREFIX}\"
VERSION                  = \"{VERSION}\"
ACTOR_NAME               = \"{ACTOR_NAME}\"
ACTOR_CATEGORY           = \"{ACTOR_CATEGORY}\"

BUCKET_NAME              = \"{BUCKET_NAME}\"
EMBEDDING_DIR_BUCKET     = \"{EMBEDDING_DIR_BUCKET}\"

BUCKET_URI               = \"{BUCKET_URI}\"
EMBEDDING_DIR_BUCKET_URI = \"{EMBEDDING_DIR_BUCKET_URI}\"

VPC_NETWORK_FULL         = \"{VPC_NETWORK_FULL}\"

ME_INDEX_NAME            = \"{ME_INDEX_NAME}\"
ME_INDEX_ENDPOINT_NAME   = \"{ME_INDEX_NAME}_endpoint\"
ME_DIMENSIONS            = \"{ME_DIMENSIONS}\"

MY_BQ_DATASET            = \"{MY_BQ_DATASET}\"
MY_BQ_TRENDS_DATASET     = \"{MY_BQ_TRENDS_DATASET}\"
"""
print(config)


PROJECT_ID               = "wortz-project-352116"
PROJECT_NUM              = "679926387543"
LOCATION                 = "us-central1"

REGION                   = "us-central1"
BQ_LOCATION              = "US"
VPC_NETWORK_NAME         = "me-network"

CREATE_NEW_ASSETS        = "True"
ACTOR_PREFIX             = "ggl"
VERSION                  = "v1"
ACTOR_NAME               = "google"
ACTOR_CATEGORY           = "technology"

BUCKET_NAME              = "zghost-ggl-v1-wortz-project-352116"
EMBEDDING_DIR_BUCKET     = "zghost-ggl-v1-wortz-project-352116-emd-dir"

BUCKET_URI               = "gs://zghost-ggl-v1-wortz-project-352116"
EMBEDDING_DIR_BUCKET_URI = "gs://zghost-ggl-v1-wortz-project-352116-emd-dir"

VPC_NETWORK_FULL         = "projects/679926387543/global/networks/me-network"

ME_INDEX_NAME            = "vectorstore_ggl_v1"
ME_INDEX_ENDPOINT_NAME   = "vectorstore_ggl_v1_endpoint"
ME_DIMENSIONS            = "768"

MY_BQ_DATASET            = "zghost_ggl_v1"
MY_BQ_TRENDS_DATASET     = "zg

In [15]:
!echo '{config}' | gsutil cp - {BUCKET_URI}/config/notebook_env.py

Copying from <STDIN>...
/ [1 files][    0.0 B/    0.0 B]                                                
Operation completed over 1 objects.                                              


List out the bucket contents and verify access

In [16]:
!gsutil ls $BUCKET_URI

gs://zghost-ggl-v1-wortz-project-352116/config/
gs://zghost-ggl-v1-wortz-project-352116/pipeline_root/


### Make a copy of the Google Trends Public Dataset

In the `plan-and-execute` agents notebook, we will provide this public dataset as a tool in its information retrieval. Because we can't run an agent directly against a public dataset (permissions), let's make a copy of that dataset into our newly created BigQuery dataset

![](imgs/public_trends_data.png)

In [64]:
BQ_PUBLIC_DATA_PROJECT       = 'bigquery-public-data'
TRENDS_DATASET               = 'google_trends'
INTL_TOP_RISING_TRENDS_TABLE = 'international_top_rising_terms'
INTL_TOP_TRENDS_TABLE        = 'international_top_terms'
TOP_RISING_TRENDS            = 'top_rising_terms'
TOP_TERMS_TABLE              = 'top_terms'

Create the international top rising trends table - you should see the print out containing information around how long the job took to run after it completes.

In [65]:
if CREATE_NEW_ASSETS:
    # here
    query = f"""
        CREATE OR REPLACE TABLE `{PROJECT_ID}.{MY_BQ_TRENDS_DATASET}.{INTL_TOP_RISING_TRENDS_TABLE}` AS (
            SELECT * FROM `{BQ_PUBLIC_DATA_PROJECT}.{TRENDS_DATASET}.{INTL_TOP_RISING_TRENDS_TABLE}`
        )
    """
    print(query)
    job = bqclient.query(query)
    job.result()
    print(job.state, (job.ended-job.started).total_seconds())


        CREATE OR REPLACE TABLE `wortz-project-352116.zghost_way_v1_trends.international_top_rising_terms` AS (
            SELECT * FROM `bigquery-public-data.google_trends.international_top_rising_terms`
        )
    
DONE 10.209


Create the international top trends table - you should see the print out containing information around how long the job took to run after it completes.

In [66]:
if CREATE_NEW_ASSETS:
    query = f"""
        CREATE OR REPLACE TABLE `{PROJECT_ID}.{MY_BQ_TRENDS_DATASET}.{INTL_TOP_TRENDS_TABLE}` AS (
            SELECT * FROM `{BQ_PUBLIC_DATA_PROJECT}.{TRENDS_DATASET}.{INTL_TOP_TRENDS_TABLE}`
        )
    """
    # print(query)
    job = bqclient.query(query)
    job.result()
    print(job.state, (job.ended-job.started).total_seconds())

DONE 8.04


Create the top rising trends table - you should see the print out containing information around how long the job took to run after it completes.

In [67]:
if CREATE_NEW_ASSETS:
    query = f"""
        CREATE OR REPLACE TABLE `{PROJECT_ID}.{MY_BQ_TRENDS_DATASET}.{TOP_RISING_TRENDS}` AS (
            SELECT * FROM `{BQ_PUBLIC_DATA_PROJECT}.{TRENDS_DATASET}.{TOP_RISING_TRENDS}`
        )
    """
    # print(query)
    job = bqclient.query(query)
    job.result()
    print(job.state, (job.ended-job.started).total_seconds())

DONE 5.97


Create the top terms trends table - you should see the print out containing information around how long the job took to run after it completes.

In [ ]:
if CREATE_NEW_ASSETS:
    query = f"""
        CREATE OR REPLACE TABLE `{PROJECT_ID}.{MY_BQ_TRENDS_DATASET}.{TOP_TERMS_TABLE}` AS (
            SELECT * FROM `{BQ_PUBLIC_DATA_PROJECT}.{TRENDS_DATASET}.{TOP_TERMS_TABLE}`
        )
    """
    # print(query)
    job = bqclient.query(query)
    job.result()
    print(job.state, (job.ended-job.started).total_seconds())